## Deploy custom container to AML endpoint

In [1]:
# import required libraries
from azure.ai.ml import MLClient
from azure.ai.ml.entities import (
    ManagedOnlineEndpoint,
    ManagedOnlineDeployment,
    Model,
    Environment,
    CodeConfiguration,
)
import os
from pathlib import Path
from dotenv import load_dotenv
from azure.identity import DefaultAzureCredential, ClientSecretCredential

In [2]:
load_dotenv

<function dotenv.main.load_dotenv(dotenv_path: Union[str, ForwardRef('os.PathLike[str]'), NoneType] = None, stream: Optional[IO[str]] = None, verbose: bool = False, override: bool = False, interpolate: bool = True, encoding: Optional[str] = 'utf-8') -> bool>

In [3]:
AZURE_TENANT_ID = os.environ.get("AZURE_TENANT_ID")
AZURE_SUBSCRIPTION_ID = os.environ.get("AZURE_SUBSCRIPTION_ID")
AZURE_CLIENT_ID = os.environ.get("AZURE_CLIENT_ID")
AZURE_CLIENT_SECRET = os.environ.get("AZURE_CLIENT_SECRET")
AZURE_ML_RESOURCE_GROUP = os.environ.get("AZURE_ML_RESOURCE_GROUP")
AZURE_ML_WORKSPACE = os.environ.get("AZURE_ML_WORKSPACE")

PROJECT_PATH = os.environ.get("PROJECT_PATH")
BASE_PATH = os.path.join(PROJECT_PATH, os.environ.get("BASE_PATH"))
AML_MODEL_NAME = os.environ.get("AML_MODEL_NAME")
ENDPOINT_MODEL_NAME = os.environ.get("ENDPOINT_MODEL_NAME")
MODEL_BASE_PATH = os.environ.get("MODEL_BASE_PATH")

print(BASE_PATH)
# endpoint_path = os.path.join(PROJECT_PATH, BASE_PATH)
model_path = os.path.join(MODEL_BASE_PATH, AML_MODEL_NAME, "1")
model_path

/workspaces/mlflow_server/online_endpoint


'/var/azureml-app/azureml-models/tfserving-mounted/1'

In [4]:
credential = DefaultAzureCredential()

credential = ClientSecretCredential(
    tenant_id=AZURE_TENANT_ID,
    client_id=AZURE_CLIENT_ID,
    client_secret=AZURE_CLIENT_SECRET,
)

ml_client = MLClient(
    credential, AZURE_SUBSCRIPTION_ID, AZURE_ML_RESOURCE_GROUP, AZURE_ML_WORKSPACE
)

In [9]:
# Creating a unique endpoint name with current datetime to avoid conflicts
import datetime

model_name="dev.ml_team.iris_classifier"
model_version=1
base_image="mlflow"
image=f"{base_image}_{model_name}_v{model_version}"
version="2024.10.1.dev20241018T2105"
image_name=f"{image}:{version}"
online_endpoint_name = f"endpoint-" + datetime.datetime.now().strftime("%m%d%H%M%f")

print(online_endpoint_name)

# create an online endpoint
endpoint = ManagedOnlineEndpoint(
    name=online_endpoint_name,
    description=f"Enpoint for {image_name}",
    auth_mode="key",
    tags={"model_name": model_name, "model_version": model_version, "image_name": image, "image_version": version},
)

endpoint-10182330108685


In [ ]:
# Create the endpoint
ml_client.begin_create_or_update(endpoint).result()

In [11]:
datetime.datetime.now().strftime("%Y%m%d%H%M%f")

'202410182249986610'